In [ ]:
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d

# Load datasets
geo_ds = xr.open_dataset(nc_files[0], group='geophysical_data')
nav_ds = xr.open_dataset(nc_files[0], group='navigation_data')

# Extract control point info
lat_ctl = nav_ds['latitude'].data  # (number_of_lines, pixel_control_points)
lon_ctl = nav_ds['longitude'].data
rows = nav_ds['cntl_pt_rows'].data  # (number_of_lines,) usually just np.arange(n_lines)
cols = nav_ds['cntl_pt_cols'].data  # (pixel_control_points,)

n_lines, n_pixels = geo_ds.dims['number_of_lines'], geo_ds.dims['pixels_per_line']

# Interpolate longitude and latitude across pixel dimension for each line
lat_full = np.empty((n_lines, n_pixels), dtype=np.float32)
lon_full = np.empty((n_lines, n_pixels), dtype=np.float32)

for i in range(n_lines):
    f_lat = interp1d(cols, lat_ctl[i, :], bounds_error=False, fill_value="extrapolate")
    f_lon = interp1d(cols, lon_ctl[i, :], bounds_error=False, fill_value="extrapolate")
    lat_full[i, :] = f_lat(np.arange(n_pixels))
    lon_full[i, :] = f_lon(np.arange(n_pixels))

# Assign to geo dataset
geo_ds = geo_ds.assign_coords(
    lat=(('number_of_lines', 'pixels_per_line'), lat_full),
    lon=(('number_of_lines', 'pixels_per_line'), lon_full)
)